In [1]:
import sys, os
sys.path.append('../')  # add parent directory to the system path
from utils_data import *
from utils_model import *
from utils_model_inference import *

import os
import cv2
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader

parent_path = "Research"

In [2]:
# ________________ MAKE TRAIN DATA READY FOR TRAINING ________________ #
train_path, _, test_path = make_data_path(parent_path)

path_to_train_JSON = os.path.join(train_path, "train_images", "train.json")
path_to_train_images = os.path.join(train_path, "train_images")
register_coco_instances("train", {}, path_to_train_JSON, path_to_train_images)
train_metadata = MetadataCatalog.get("train")
train_dataset_dicts = DatasetCatalog.get("train")

path_to_test_JSON = os.path.join(test_path, "test_images", "test.json")
path_to_test_images = os.path.join(test_path, "test_images")
register_coco_instances("test", {}, path_to_test_JSON, path_to_test_images)
test_metadata = MetadataCatalog.get("test")
test_dataset_dicts = DatasetCatalog.get("test")


# ****** --------- ****** #
test_name = "test6"
note = "using high-contrast images"
# ****** --------- ****** #


model = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
# model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
# model = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"


cfg = get_cfg()
cfg.OUTPUT_DIR = "../models/test19/"
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = ""
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000    # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = [1000, 2000, 3000]        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # We have 4 classes.
# NOTE: this config means the number of classes, without the background. Do not use num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False) #Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available

In [3]:
trainer.train() #Start the training process

2023-10-29 01:32:31.950530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 01:32:33.377684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
